In [1]:
from ultralytics import YOLO
from tqdm.auto import tqdm
import numpy as np
import os
import pandas as pd
import cv2
from ultralytics.yolo.utils.plotting import Annotator

In [2]:
# Load a model
model = YOLO('yolov8n-pose.pt') # load an official model

In [3]:
dataset_path = "M:/Datasets/shock_walk/Videos/"
files = [i for i in os.listdir(dataset_path) if i.split('.')[-1] == "mp4"]
lengths  = []

for file in tqdm(files):
    
    skeletons = np.empty((1,17,3))
    video = cv2.VideoCapture(dataset_path + file)
    ret = True
    
    while ret:
        ret, image = video.read()
        if not ret:
            break
        
        image = image[:,300:-200]
        
        results = model(image, verbose=False)
        keypoint = results[0].keypoints.cpu()
        skeletons = np.concatenate((skeletons, keypoint.numpy()))
        
        annotator = Annotator(np.ascontiguousarray(image))
        annotator.kpts(keypoint[0])
        
        cv2.imshow("image", annotator.result())
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    frame_per_video = 30 * 60
    
    if len(skeletons) > 1000:
        skeletons = skeletons[:frame_per_video]
    
    else:
        skeletons = skeletons[:315]
    
    lengths.append(len(skeletons))
    np.save(f"{dataset_path}skeletons/{file.split('.')[0]}.npy", skeletons)
    
cv2.destroyAllWindows()

  0%|          | 0/5 [00:00<?, ?it/s]

(1920, 588, 3)


NameError: name 'sd' is not defined

In [15]:
print(lengths)

[1800, 1800, 315, 1800, 1800]
